In [19]:
from datetime import timedelta
from airflow.models import DAG
from airflow.operators.python_operator import PythonOperator
# from airflow.utils.dates import days_ago
import pandas as pd
import os
import string
from nltk.corpus import stopwords
from urllib.parse import urlparse

# Make sure to download the stopwords set if you haven't already
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Define default arguments for DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': days_ago(1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}



# Define function to preprocess data
def preprocess_data():
    # Load the data
    df = pd.read_csv('scraped_news.csv')
    
    # Function to extract domain name from URL
    def extract_domain(link):
        parsed_url = urlparse(link)
        domain = parsed_url.netloc.replace('www.', '')
        return domain.split('.')[0]

    # Function to clean text in Title and Content
    def clean_text(text):
        # Convert text to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text

    # Apply the function to extract source domain and clean text
    df['source'] = df['Links'].apply(extract_domain)
    df['Title'] = df['Title'].apply(clean_text)
    df['Content'] = df['Content'].apply(clean_text)
    
    # Save the transformed data
    df.to_csv('scraped_news_transformed.csv', index=False)

# # Execute the function to preprocess data
# preprocess_data()

# Define function to upload data to Google Drive
def upload_to_gdrive():
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    
    # Authenticate and create the PyDrive client
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    
    # Create and upload a text file
    file1 = drive.CreateFile({'title': 'scraped_news_transformed.csv'})
    file1.SetContentFile('scraped_news_transformed.csv')
    file1.Upload()
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))

# Define function to manage DVC
def update_dvc():
    # Add the transformed data file to DVC tracking
    os.system('dvc add scraped_news_transformed.csv')
    # Commit the changes to the DVC repository
    os.system('git add .dvc/config scraped_news_transformed.csv.dvc')
    os.system('git commit -m "Update data processed by Airflow DAG"')
    # Push the changes
    os.system('dvc push')
    os.system('git push')

# Set up the DAG
with DAG(
    'data_processing_pipeline',
    default_args=default_args,
    description='DAG for processing and managing data using DVC and Google Drive.',
    schedule_interval=timedelta(days=1),
    catchup=False,
) as dag:
    
    # Define tasks
    t1 = PythonOperator(
        task_id='preprocess_data',
        python_callable=preprocess_data,
    )
    
    t2 = PythonOperator(
        task_id='upload_to_gdrive',
        python_callable=upload_to_gdrive,
    )
    
    t3 = PythonOperator(
        task_id='update_dvc',
        python_callable=update_dvc,
    )
    
    # Define task dependenciesA
    t1 >> t2 >> t3



c:\Users\Faizan\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow\__init__.py:35 RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.

[2024-05-09T18:07:46.576+0500] {logging_config.py:71} ERROR - Unable to load the config, contains a configuration error.


ValueError: Unable to configure formatter 'airflow'

In [20]:
%pip install dag


  Using cached DAG-0.tar.gz (5.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Faizan\AppData\Local\Temp\pip-install-ssinhdlt\dag_cf983bea5bd2404fa97566e6ea818748\setup.py", line 1, in <module>
          from ez_setup import use_setuptools
        File "C:\Users\Faizan\AppData\Local\Temp\pip-install-ssinhdlt\dag_cf983bea5bd2404fa97566e6ea818748\ez_setup.py", line 94
          except pkg_resources.VersionConflict, e:
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      SyntaxError: multiple exception types must be parenthesized
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: